In [1]:
from lsst.daf.butler import Butler

In [2]:
butler = Butler("/project/hchiang2/ci_hsc_gen3/w_2019_43/ci_hsc_gen3/DATA", collection="shared/ci_hsc_output")

In [183]:
butler.registry.getAllCollections()

{'calib/hsc',
 'masks/hsc',
 'raw/hsc',
 'ref_cats',
 'shared/ci_hsc_output',
 'skymaps'}

The API is very similar to the Gen2.  We can `butler.get` with a dict of data IDs like before

In [184]:
exp = butler.get("calexp", {"visit":903334, "detector":22, "instrument":"HSC"})
print(exp.getWcs())
wcs = butler.get("calexp.wcs", {"visit":903334, "detector":22, "instrument":"HSC"})
print(wcs)
vinfo = butler.get("calexp.visitInfo", {"visit":903334, "detector":22, "instrument":"HSC"})
print(vinfo)

FITS standard SkyWcs:
Sky Origin: (320.921403, -0.286257)
Pixel Origin: (934.627, 2742.18)
Pixel Scale: 0.158394 arcsec/pixel
FITS standard SkyWcs:
Sky Origin: (320.921403, -0.286257)
Pixel Origin: (934.627, 2742.18)
Pixel Scale: 0.158394 arcsec/pixel
VisitInfo(exposureId=180666822, exposureTime=30, darkTime=30, date=2013-06-17T13:29:27.267000000, UT1=nan, ERA=1.88512 rad, boresightRaDec=(320.249933, +0.000019), boresightAzAlt=(157.958889, +68.807106), boresightAirmass=1.07239, boresightRotAngle=4.71239 rad, rotType=1, observatory=-155.477W, 19.8256N  4139, weather=Weather(0.6, 62130, 72.5))


We can get all data IDs/Dimensions. 
Note that `ref.dataId` is no longer a simple dict; it's a `ExpandedDataCoordinate`

In [185]:
for ref in butler.registry.queryDatasets("calexp", collections=['shared/ci_hsc_output']):
    print(ref.dataId)

{'instrument': 'HSC', 'detector': 22, 'visit': 903334}
{'instrument': 'HSC', 'detector': 16, 'visit': 903334}
{'instrument': 'HSC', 'detector': 16, 'visit': 903986}
{'instrument': 'HSC', 'detector': 5, 'visit': 903344}
{'instrument': 'HSC', 'detector': 10, 'visit': 904010}
{'instrument': 'HSC', 'detector': 100, 'visit': 904010}
{'instrument': 'HSC', 'detector': 16, 'visit': 903988}
{'instrument': 'HSC', 'detector': 100, 'visit': 903986}
{'instrument': 'HSC', 'detector': 23, 'visit': 903986}
{'instrument': 'HSC', 'detector': 100, 'visit': 903334}
{'instrument': 'HSC', 'detector': 18, 'visit': 903338}
{'instrument': 'HSC', 'detector': 12, 'visit': 903346}
{'instrument': 'HSC', 'detector': 6, 'visit': 903346}
{'instrument': 'HSC', 'detector': 4, 'visit': 903342}
{'instrument': 'HSC', 'detector': 17, 'visit': 903336}
{'instrument': 'HSC', 'detector': 6, 'visit': 904014}
{'instrument': 'HSC', 'detector': 4, 'visit': 904010}
{'instrument': 'HSC', 'detector': 0, 'visit': 903344}
{'instrument'

In Gen3, we can also get the WCS and the file URI without dumping the images as Python objects, for example

In [191]:
for ref in butler.registry.queryDatasets("calexp.wcs", collections=['shared/ci_hsc_output']):
    wcs = butler.get(ref)
    uri = butler.datastore.getUri(ref)
    print("calexp has ", wcs, "\nand the file is at \n", uri)

calexp has  FITS standard SkyWcs:
Sky Origin: (320.921403, -0.286257)
Pixel Origin: (934.627, 2742.18)
Pixel Scale: 0.158394 arcsec/pixel 
and the file is at 
 file:///project/hchiang2/ci_hsc_gen3/w_2019_43/ci_hsc_gen3/DATA/shared/ci_hsc_output/calexp/r/HSC-R/903334/calexp_r_HSC-R_903334_22_HSC_7.fits
calexp has  FITS standard SkyWcs:
Sky Origin: (320.746067, -0.390681)
Pixel Origin: (1033.9, 2212.6)
Pixel Scale: 0.161463 arcsec/pixel 
and the file is at 
 file:///project/hchiang2/ci_hsc_gen3/w_2019_43/ci_hsc_gen3/DATA/shared/ci_hsc_output/calexp/r/HSC-R/903334/calexp_r_HSC-R_903334_16_HSC_7.fits
calexp has  FITS standard SkyWcs:
Sky Origin: (320.745734, -0.391748)
Pixel Origin: (1049.18, 2208.14)
Pixel Scale: 0.16141 arcsec/pixel 
and the file is at 
 file:///project/hchiang2/ci_hsc_gen3/w_2019_43/ci_hsc_gen3/DATA/shared/ci_hsc_output/calexp/i/HSC-I/903986/calexp_i_HSC-I_903986_16_HSC_7.fits
calexp has  FITS standard SkyWcs:
Sky Origin: (320.801904, -0.331817)
Pixel Origin: (1074.12, 

With the `DatasetRef`, we may also use `butler.getDirect`

In [187]:
rows = butler.registry.queryDatasets("calexp", collections=['shared/ci_hsc_output'])
ref = list(rows)[0]  # Just to get the first DatasetRef
exp = butler.getDirect(ref)
exp.getWcs()

FITS standard SkyWcs:
Sky Origin: (320.921403, -0.286257)
Pixel Origin: (934.627, 2742.18)
Pixel Scale: 0.158394 arcsec/pixel

In [3]:
import lsst.geom as geom

In [4]:
for ref in butler.registry.queryDatasets("calexp", collections=['shared/ci_hsc_output']):#, where="detector = 22"):
    uri = butler.datastore.getUri(ref)
    print("==== For the file of ", ref.dataId, "at \n", uri)
    exp = butler.getDirect(ref)
    wcs = exp.getWcs()

    print("dimensions:", exp.getDimensions())
    print("pixel scale:", wcs.getPixelScale().asArcseconds())
    
    imageBox = geom.Box2D(exp.getBBox())
    corners = [wcs.pixelToSky(pix) for pix in imageBox.getCorners()]
    imageCenter = wcs.pixelToSky(imageBox.getCenter())
    
    print("ra and dec for the center:", imageCenter.getRa().asDegrees(), imageCenter.getDec().asDegrees())
    print("ra and dec for the corners:")
    [print(corner) for corner in corners]

==== For the file of  {'instrument': 'HSC', 'detector': 22, 'visit': 903334} at 
 file:///project/hchiang2/ci_hsc_gen3/w_2019_43/ci_hsc_gen3/DATA/shared/ci_hsc_output/calexp/r/HSC-R/903334/calexp_r_HSC-R_903334_22_HSC_7.fits
dimensions: (2048, 4176)
pixel scale: 0.15839415126949158
ra and dec for the center: 320.94931370489684 -0.28952700960859995
ra and dec for the corners:
(321.038074, -0.241236)
(321.035198, -0.332304)
(320.858064, -0.337951)
(320.860040, -0.245224)
==== For the file of  {'instrument': 'HSC', 'detector': 16, 'visit': 903334} at 
 file:///project/hchiang2/ci_hsc_gen3/w_2019_43/ci_hsc_gen3/DATA/shared/ci_hsc_output/calexp/r/HSC-R/903334/calexp_r_HSC-R_903334_16_HSC_7.fits
dimensions: (2048, 4176)
pixel scale: 0.16146344831556486
ra and dec for the center: 320.7516564075301 -0.39008487855662477
ra and dec for the corners:
(320.845034, -0.341677)
(320.842442, -0.433390)
(320.656949, -0.438273)
(320.658582, -0.345423)
==== For the file of  {'instrument': 'HSC', 'detector